# 04 — Writing a Mini Literature Review with RAG

**What you'll do in this notebook**

In the previous notebook, you curated and saved a *top‑K* set of papers to a CSV file (`literature_corpus_top.csv`).  
Here, you will:

1. **Load** the curated corpus (already filtered to the top‑K papers).  
2. **Ask a research question** to focus your review.  
3. Run a **simple keyword search** (for quick, transparent validation).  
4. Ask an LLM to write a **two‑paragraph literature review**, with **real citations** that link to each paper's **PDF**.  
5. Read a short **theoretical note** on more sophisticated approaches for larger corpora.

> This notebook is designed for **entry-level researchers**. It favours clarity and minimal code over complexity.


## 0) Groq API Settings (Run first)

Paste your Groq API key. This is **not secure** if you share the notebook—it's fine for a workshop demo.


In [ ]:
# --- Groq API Settings ---
import os

# Paste your Groq API key below (between quotes).
# NOTE: This is not a secure way of entering API keys if you share the notebook.
os.environ["GROQ_API_KEY"] = ""  # <-- replace with your key (e.g., "gsk_...")
model = "llama-3.3-70b-versatile"  # Pick a model. See https://console.groq.com/docs/models
os.environ["BASE_URL"] = "https://api.groq.com/openai/v1"  # OpenAI-compatible API
print("Groq settings configured. Remember to paste your API key!")    


Groq settings configured. Remember to paste your API key!


## 1) Create the client and the `chat()` function

We use a single helper function `chat()` to talk to the model.  
Two convenience helpers manage the conversation memory:

- `set_system(text)`: sets the system prompt and resets the history.  
- `clear_chat()`: clears history entirely.

> Tip: Resetting the chat between tasks keeps context clean and results reproducible.


In [2]:
# --- Create the OpenAI-compatible client + chat() helper ---
from openai import OpenAI

chat_history = []  # running memory for the current conversation

client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url=os.environ["BASE_URL"],
)

def set_system(prompt: str):
    """Reset chat history to a single system message."""
    global chat_history
    chat_history = [{"role": "system", "content": prompt}]
    return chat_history

def clear_chat():
    """Clear all chat history."""
    global chat_history
    chat_history = []
    return chat_history

def chat(user_input, temperature: float = 0.2, max_turns: int = 8):
    """Send one user turn and get a reply, preserving context.
    - Keeps system prompt + last `max_turns` user/assistant messages.
    """
    global chat_history
    chat_history.append({"role": "user", "content": user_input})

    # Keep only the most recent `max_turns` pairs to control context size
    system = chat_history[:1]
    recent = chat_history[-(max_turns*2):] if len(chat_history) > 1 else []
    window = system + recent

    resp = client.chat.completions.create(
        model=model,
        messages=window,
        temperature=temperature,
    )
    reply = resp.choices[0].message.content
    chat_history.append({"role": "assistant", "content": reply})
    return reply

print("Testing API ...")
print()
print("User:")
print("Hello LLM!")
print("LLM:")
print(chat("Hello LLM!"))
print()
print("Client ready. Use chat('...'), set_system('...'), clear_chat().")


Testing API ...

User:
Hello LLM!
LLM:
Hello! It's nice to meet you. Is there something I can help you with or would you like to chat?

Client ready. Use chat('...'), set_system('...'), clear_chat().


## 2) Load the curated corpus (top‑K)

In Notebook **03**, you created `literature_corpus_top.csv`—a curated set of the most relevant papers.  
We load it here and briefly inspect the columns. You can open the CSV later in Excel if you like.

> **Important:** We will use **every row** in this CSV (no further filtering here), because this file already represents the *top‑K* selection.


In [3]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path("literature_data")
CORPUS_CSV = DATA_DIR / "literature_corpus_top.csv"
assert CORPUS_CSV.exists(), "Could not find literature_corpus_top.csv. Run Notebook 03 first."

corpus = pd.read_csv(CORPUS_CSV)
print(f"Loaded corpus with {len(corpus)} rows.")
corpus.head(3)


Loaded corpus with 3 rows.


,title,summary,published,link,pdf,authors,summary_llm,structured,pdf_file,crude_text
0,Local-projective-measurement-enhanced quantum ...,Quantum batteries have significant potential a...,2024-05-06,NaN,http://arxiv.org/pdf/2405.03093v1,"Tinggui Zhang, Hong Yang, Shao-Ming Fei",This study explores methods to enhance the cap...,{'methodology': 'Local projective measurements...,2405.03093v1.pdf,arXiv:2405.03093v1 [quant-ph] 6 May 2024\nLo...
1,Optimizing quantum battery performance by redu...,Quantum batteries have emerged as promising de...,2025-05-12,NaN,http://arxiv.org/pdf/2505.08029v1,"Rohit Kumar Shukla, Rajiv Kumar, Ujjwal Sen, S...",Researchers investigated the dynamics of quant...,{'methodology': 'Investigating interplay betwe...,2505.08029v1.pdf,arXiv:2505.08029v1 [quant-ph] 12 May 2025\nO...
2,Efficient charging of multiple open quantum ba...,We explore a protocol that efficiently charges...,2024-10-25,NaN,http://arxiv.org/pdf/2410.19303v1,"Josephine Dias, Hui Wang, Kae Nemoto, Franco N...",This study aims to develop an efficient protoc...,{'methodology': 'protocol using collective cou...,2410.19303v1.pdf,Efficient charging of multiple open quantum ba...


*What are these columns?*  
- **title**: paper title  
- **authors**: authors (string)  
- **published**: publication date (from arXiv)  
- **pdf**: direct PDF link (we will cite this in the review)  
- **link**: arXiv HTML page (fallback if PDF is missing)  
- **summary** / **summary_llm**: abstract and/or LLM summary  
- **crude_text**: optional partial text extracted from PDF in the previous notebook

If some of these are missing, don't worry—this notebook still works with what you have.


## 3) Ask a research question

A clear question focuses the search and helps the model write a coherent review.  
You can (and should) edit this to your topic of interest.

You can also pick keywords and only work on papers that contain at least one keywrod. 


In [4]:
# 👉 Edit this question to match your topic.
research_question = "How do local projective measurements influence the charging capacity of quantum batteries?"
keywords = ["quantum battery", "measurement", "projective", "capacity", "charging"]



*Why this matters:* A good question anchors both your manual exploration (keyword checks) and the model's synthesis.


## 4) Keyword search (quick evaluation & exploration)

Before synthesising a review, it's useful to quickly scan which papers mention key terms from the question.  
This is **not** semantic search—it's a simple, transparent way to build intuition.

**What we’ll do:**
1. Choose a small list of keywords (derived from the question).  
2. Search each paper’s `summary` (or `summary_llm` / `crude_text` if `summary` is missing).  
3. Display papers that mention at least one keyword.


In [5]:
# Choose simple keywords (edit freely)
# Get all text from the crude_text column (empty string if missing)
text_series = corpus["crude_text"].fillna("").astype(str).str.lower()

# Check whether each text contains any of the keywords
mask = text_series.apply(lambda txt: any(k in txt for k in keywords))

# Keep only rows that contain at least one keyword
keyword_hits = corpus[mask].copy()

keyword_hits.head(10)


,title,summary,published,link,pdf,authors,summary_llm,structured,pdf_file,crude_text
0,Local-projective-measurement-enhanced quantum ...,Quantum batteries have significant potential a...,2024-05-06,NaN,http://arxiv.org/pdf/2405.03093v1,"Tinggui Zhang, Hong Yang, Shao-Ming Fei",This study explores methods to enhance the cap...,{'methodology': 'Local projective measurements...,2405.03093v1.pdf,arXiv:2405.03093v1 [quant-ph] 6 May 2024\nLo...
1,Optimizing quantum battery performance by redu...,Quantum batteries have emerged as promising de...,2025-05-12,NaN,http://arxiv.org/pdf/2505.08029v1,"Rohit Kumar Shukla, Rajiv Kumar, Ujjwal Sen, S...",Researchers investigated the dynamics of quant...,{'methodology': 'Investigating interplay betwe...,2505.08029v1.pdf,arXiv:2505.08029v1 [quant-ph] 12 May 2025\nO...
2,Efficient charging of multiple open quantum ba...,We explore a protocol that efficiently charges...,2024-10-25,NaN,http://arxiv.org/pdf/2410.19303v1,"Josephine Dias, Hui Wang, Kae Nemoto, Franco N...",This study aims to develop an efficient protoc...,{'methodology': 'protocol using collective cou...,2410.19303v1.pdf,Efficient charging of multiple open quantum ba...


*Reading the results:* Hit counts provide a rough, explainable signal.  
If you see very few hits, consider broadening your keywords or checking a different text column (e.g., `crude_text`).


## 5) Write a two‑paragraph literature review (with real links)

We now ask the LLM to write a **two‑paragraph** literature review that directly answers our research question.  
To keep the output **verifiable**, we ask the model to cite papers by **linking to their PDF** (from the `pdf` column).  

**How we make it reliable:**
- We present the model with the **full table** of candidate papers (already curated top‑K).  
- We give a **clear instruction** to base the review *only* on these papers.  
- We explicitly ask for citations using the `pdf` links in parentheses.


In [6]:
# We build a compact textual view of the corpus that the model can read.
# Keep the table short enough to fit into the prompt. If your corpus is long,
# consider slicing columns or truncating summaries.

subset_cols = [c for c in ["title","summary","pdf"] if c in corpus.columns]
table_for_prompt = corpus[subset_cols].fillna("").copy()

# Truncate long summaries to keep the prompt smaller and cheaper
table_for_prompt["summary"] = table_for_prompt["summary"].apply(lambda s: (s[:700] + "…") if isinstance(s, str) and len(s) > 700 else s)
table_for_prompt

,title,summary,pdf
0,Local-projective-measurement-enhanced quantum ...,Quantum batteries have significant potential a...,http://arxiv.org/pdf/2405.03093v1
1,Optimizing quantum battery performance by redu...,Quantum batteries have emerged as promising de...,http://arxiv.org/pdf/2505.08029v1
2,Efficient charging of multiple open quantum ba...,We explore a protocol that efficiently charges...,http://arxiv.org/pdf/2410.19303v1


In [7]:

prompt = f"""    Write a coherent **two-paragraph literature review** answering the question below.
Use **only** the papers provided. For each claim, cite at least one paper
by adding the paper's **PDF link** in parentheses, e.g., (https://arxiv.org/pdf/XXXX).

**Question:**
{research_question}

**Papers (title, summary, pdf):**
{table_for_prompt.to_string(index=False)}

Requirements:
- Two paragraphs, formal academic tone
- Factual, concise, and grounded in the provided papers
- Each paragraph should contain at least one citation using the `pdf` link
- Do not invent citations or links
"""

prompt

"    Write a coherent **two-paragraph literature review** answering the question below.\nUse **only** the papers provided. For each claim, cite at least one paper\nby adding the paper's **PDF link** in parentheses, e.g., (https://arxiv.org/pdf/XXXX).\n\n**Question:**\nHow do local projective measurements influence the charging capacity of quantum batteries?\n\n**Papers (title, summary, pdf):**\n                                                                                    title                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [8]:

clear_chat()
set_system("You are an academic assistant writing short literature reviews.")

review_text = chat(prompt, temperature=0.3)
print(review_text)


The influence of local projective measurements on the charging capacity of quantum batteries has been a subject of interest in recent studies. Research has shown that local projective measurements can enhance the capacity of quantum batteries, particularly in bipartite quantum systems (http://arxiv.org/pdf/2405.03093v1). By applying local projective measurements on a subsystem of the quantum state, the battery capacity with respect to the whole system or a subsystem can be improved. This has been demonstrated using two-qubit Bell-diagonal states and X-type states as examples, highlighting the potential of local projective measurements in optimizing quantum battery performance (http://arxiv.org/pdf/2405.03093v1). Furthermore, the interplay between the battery and charger Hamiltonians has also been investigated, with a focus on minimizing the battery's influence during the charging dynamics (http://arxiv.org/pdf/2505.08029v1).

The optimization of quantum battery performance is a complex

*Tip:* If you see missing or incorrect links, check your `pdf` column values.  
You can also reduce the prompt size by limiting columns, or by shortening summaries more aggressively.


## 6) Save the generated review (optional but recommended)

Saving the output makes it easy to share or iterate later.


In [9]:
from pathlib import Path
out_txt = Path("literature_data") / "mini_lit_review.txt"
with open(out_txt, "w", encoding="utf-8") as f:
    f.write(review_text)
print("Saved review to:", out_txt.resolve())


Saved review to: C:\code\llm_tutorial\literature_data\mini_lit_review.txt


## 7) Beyond this notebook — methods for larger corpora (theory only)

When the number of papers becomes large, we need methods that scale while keeping quality high.

**A. Embedding‑based retrieval and clustering**  
- Convert each abstract/full text and your question into **vector embeddings**.  
- Use cosine similarity to retrieve the most relevant papers.  
- Cluster embeddings (e.g., with K‑Means or HDBSCAN) to reveal **themes** or **subtopics**, then summarise per cluster.

**B. Hierarchical summarisation**  
- Summarise small batches (or clusters) of papers first.  
- Then summarise those summaries to produce a clean, high‑level narrative.  
- This *divide‑and‑conquer* approach scales to hundreds of papers.

**C. Hybrid RAG pipelines**  
- Combine embeddings for retrieval (fast, consistent) with an LLM for synthesis (nuanced, fluent).  
- Ground the LLM in retrieved passages to reduce hallucinations, and always **cite** source papers.

**D. Multi‑criteria scoring**  
- Define simple criteria (e.g., *method novelty*, *data similarity*, *domain fit*, *practicality*) and score each paper (1–5).  
- Take a weighted average to rank papers. This improves transparency and repeatability in reviews.

> These ideas extend the simple approach in this notebook and are commonly used in systematic reviews and large‑scale literature surveys.
